# Chess Move PGN Tracking

In [ ]:
!pip install -U ultralytics==8.3.40 --quiet

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import csv
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import re
import shutil
import ultralytics
import warnings
from IPython.display import Video
try:
    from PIL import Image
except ImportError:
    import Image
from ultralytics import YOLO

In [ ]:
warnings.filterwarnings("ignore")

## Hand Detection

In [ ]:
def detect_hand(model_hand, image):
    results = model_hand.predict(
        image,
        imgsz=288,
        conf=0.5,
        classes=0 # Dectect only person class
    )

    # Access results for the first image
    result_0 = results[0]

    # Get bounding boxes
    boxes = result_0.boxes  # Contains bounding boxes, confidences, and class IDs

    hand_detection = False
    if boxes is None or len(boxes) == 0:
        print("No hand detection found")
    else:
        for box in boxes:
            # Get the box with the highest confidence
            best_box = boxes[0]

            # Extract bounding box details
            xmin, ymin, xmax, ymax = best_box.xyxy.cpu().numpy().flatten()
            conf = best_box.conf.cpu().numpy().item()
            # cls = best_box.cls.cpu().numpy().item()

            area_detected_hand = (xmax - xmin) * (ymax - ymin)
            cutoff_area = (1080 * 0.2) * (1080 * 0.2) # 20% of Original Image Size
            if area_detected_hand >= cutoff_area:
                # Visualize the selected box on the image
                test_image = result_0.plot()
                image_rgb = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
                plt.imshow(image_rgb)
                plt.axis("off")
                plt.show()
                print(f"Hand detection found with {conf:.4f}")
                hand_detection = True
            else:
                print("No hand detection found")

    return hand_detection

## Chess Tracking

### Get image persepective and centroid of board

In [ ]:
class ImageProcessor:
    def __init__(self, image_paths, block_size=41, C=2):
        """
        Initialize the image processor.

        Args:
            image_paths (list): List of image file paths.
            block_size (int): Block size for adaptive thresholding.
            C (int): Constant to subtract for adaptive thresholding.
        """
        self.image_paths = image_paths
        self.block_size = block_size
        self.C = C
        self.titles = [f'Board {i}' for i in range(1, len(image_paths) + 1)]

    def process_images(self):
        """Processes and displays all images."""
        plt.figure(figsize=(12, 8))
        for i, path in enumerate(self.image_paths):
            img = self._read_image(path)
            scanned = self._process_single_image(img)
            if scanned is not None:
                plt.subplot(3, 4, i + 1)
                plt.title(self.titles[i])
                plt.imshow(scanned, cmap="gray")
                plt.axis("off")
        plt.show()

    def _read_image(self, path):
        """
        Reads an image from the given path.

        Args:
            path (str): Path to the image file.

        Returns:
            img (numpy.ndarray): The read image.
        """
        img = cv2.imread(path)
        if img is None:
            print(f"Error reading image: {path}")
            return None, None
        return img

    def _process_single_image(self, img, return_with_grid=True):
        """
        Processes a single image in memory.

        Args:
            img (numpy.ndarray): The input image.

        Returns:
            numpy.ndarray: The processed and warped image, or None if processing fails.
        """
        orig = img.copy()
        if img is None or orig is None:
            print("Invalid image data provided.")
            return None

        img_RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        gray = cv2.cvtColor(img_RGB, cv2.COLOR_RGB2GRAY)
        thresholded = self._threshold_image(gray)
        edges = cv2.adaptiveThreshold(
            thresholded, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, self.block_size, self.C
        )
        contours = self._find_contours(edges)

        approxs = self._filter_quadrilaterals(contours)
        points = self._extract_points(approxs)
        if points is None or points.size == 0:
            return None

        paper_contour = self._find_paper_contour(contours)
        if paper_contour is not None:
            M, warped, complete_grid = self._warp_perspective(orig, paper_contour, points)

            if return_with_grid:
                return self._draw_grid(warped, complete_grid)
            return M, warped, complete_grid

        return None

    def _threshold_image(self, gray_image):
        """Applies thresholding to an image."""
        _, thresholded = cv2.threshold(gray_image, 190, 255, cv2.THRESH_BINARY)
        return thresholded

    def _find_contours(self, edges):
        """Finds contours in the thresholded edges."""
        contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        return contours

    def _filter_quadrilaterals(self, contours):
        """Filters quadrilateral contours based on area."""
        approxs = []
        for cnt in contours:
            epsilon = 0.02 * cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, epsilon, True)
            if len(approx) == 4 and cv2.contourArea(approx) > 4000:
                approxs.append(approx)
        return approxs

    def _extract_points(self, approxs):
        """Extracts points from filtered quadrilaterals."""
        points = []
        for approx in approxs:
            point_max = approx.max((0, 1))
            point_min = approx.min((0, 1))
            rect = point_max - point_min
            area = rect[0] * rect[1]
            if area < 50000:
                point = approx.mean((0, 1))
                points.append(point)
        return np.stack(points, axis=0) if points else np.array([])

    def _find_paper_contour(self, contours):
        """Finds the first quadrilateral contour."""
        for contour in contours:
            epsilon = 0.02 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            if len(approx) == 4:
                return approx
        return None

    def _warp_perspective(self, orig, contour, points):
        """
        Warps the perspective of the detected paper contour.

        Args:
            orig (numpy.ndarray): The original image.
            contour (numpy.ndarray): The paper contour points.
            points (numpy.ndarray): Points to transform.

        Returns:
            tuple: The warped image and the complete grid of points.
        """
        rect = self._order_points(contour.reshape(4, 2))
        dst = np.array([
            [0, 0],
            [640 - 1, 0],
            [640 - 1, 640 - 1],
            [0, 640 - 1]
        ], dtype="float32")

        # Compute the perspective transform matrix
        M = cv2.getPerspectiveTransform(rect, dst)
        warped = cv2.warpPerspective(orig, M, (640, 640))

        # Transform points and generate the grid
        transformed_points = cv2.perspectiveTransform(np.expand_dims(points, 1), M)
        complete_grid = self._generate_grid(transformed_points)

        return M, warped, complete_grid

    def _draw_grid(self, warped, complete_grid):
        """
        Draws circles on the warped image for the given grid points.

        Args:
            warped (numpy.ndarray): The warped image.
            complete_grid (numpy.ndarray): The grid points to draw.

        Returns:
            numpy.ndarray: The image with circles drawn on it.
        """
        if complete_grid is not None:
            for point in complete_grid:
                cv2.circle(warped, point.round().astype(np.int32), int(round(100 * 0.3)), (255, 255, 0), 3)
        return cv2.cvtColor(warped, cv2.COLOR_BGR2RGB)


    def _order_points(self, pts):
        """Orders points in a consistent order."""
        rect = np.zeros((4, 2), dtype="float32")
        s = pts.sum(axis=1)
        rect[0] = pts[np.argmin(s)]
        rect[2] = pts[np.argmax(s)]

        diff = np.diff(pts, axis=1)
        rect[1] = pts[np.argmin(diff)]
        rect[3] = pts[np.argmax(diff)]
        return rect

    def _generate_grid(self, transformed_points):
        """Generates a grid based on transformed points."""
        x_values = transformed_points[:, 0, 0]
        y_values = transformed_points[:, 0, 1]
        x_values = self._filter_and_sort_values(x_values)
        y_values = self._filter_and_sort_values(y_values)

        if len(x_values) < 2 or len(y_values) < 2:
            return None

        x_select = self._filter_repeated_values(x_values)
        y_select = self._filter_repeated_values(y_values)

        xx, yy = np.meshgrid(x_select, y_select)
        return np.vstack([xx.ravel(), yy.ravel()]).T

    def _filter_and_sort_values(self, values):
        """Filters and sorts values within valid bounds."""
        return np.sort(values[(values > 0) & (values < 641)])

    def _filter_repeated_values(self, values):
        """Filters repeated values and removes outliers."""
        filtered = []
        counts = []
        for value in values:
            if not filtered:
                filtered.append(value)
                counts.append(1)
            else:
                diffs = np.abs(np.array(filtered) - value)
                min_idx = np.argmin(diffs)
                if diffs[min_idx] < 10:
                    counts[min_idx] += 1
                    filtered[min_idx] = (filtered[min_idx] * (counts[min_idx] - 1) + value) / counts[min_idx]
                else:
                    filtered.append(value)
                    counts.append(1)

        filtered = np.array(filtered)
        diffs = np.diff(filtered)
        mean_diff = diffs.mean()
        outliers = np.abs(diffs - mean_diff) > 20
        remove_indices = np.where(outliers)[0] + 1
        return np.delete(filtered, remove_indices)

### Get all path of frames

In [ ]:
# Function to perform natural sorting
def natural_sort(file_list):
    def try_parse_int(s):
        try:
            return int(s)
        except ValueError:
            return s

    def alphanum_key(s):
        return [try_parse_int(c) for c in re.split('([0-9]+)', s)]

    return sorted(file_list, key=alphanum_key)

def record_paths(folder_paths):
    image_paths = []

    # Correct unpacking of os.walk()
    for root, _, files in os.walk(folder_paths):
        for file in files:
            if file.endswith(".jpg"):
                image_paths.append(os.path.join(root, file))

    # Sort the paths using natural sorting
    image_paths = natural_sort(image_paths)

    return image_paths

In [ ]:
def process_and_rotate_images(model, map_name, image_path, move2_rotate_paths, threshold=8, img_size=640):
    """
    Process an image to count chess pieces and decide whether to rotate associated images in folders.

    Parameters:
        model: Trained model for object detection.
        map_name: Dictionary mapping class indices to class names.
        image_path: Path to the image to analyze.
        move2_rotate_paths: List of paths to images that may need rotation.
        threshold: The count difference threshold for deciding rotation.
        img_size: Image size for model prediction.
    """

    def rotate(folder_paths, angle=-90, scale=1.0):
        """Rotates images in the specified folder paths."""
        for path in folder_paths:
            img = cv2.imread(path)
            if img is None:
                print(f"Could not load image: {path}")
                continue

            (h, w) = img.shape[:2]
            center = (w // 2, h // 2)
            rotation_matrix = cv2.getRotationMatrix2D(center, angle, scale)
            rotated_img = cv2.warpAffine(img, rotation_matrix, (w, h))

            cv2.imwrite(path, rotated_img)  # Overwrite the original file
            print(f"Saved rotated image to: {path}")

    # Predict objects in the image
    results = model.predict(source=image_path, imgsz=img_size)
    all_boxes = results[0].boxes.xyxy.tolist()
    count_left = [0, 0]  # Count [white, black] on the left
    count_right = [0, 0]  # Count [white, black] on the right

    for i, box in enumerate(all_boxes):
        x1, y1, x2, y2 = box
        class_index = int(results[0].boxes.cls[i].item())
        chess_color = map_name.get(class_index, "unknown").split('-')[0]
        center_x = (x1 + x2) / 2

        if center_x < 520:
            if chess_color == 'white':
                count_left[0] += 1
            elif chess_color == 'black':
                count_left[1] += 1
        else:
            if chess_color == 'white':
                count_right[0] += 1
            elif chess_color == 'black':
                count_right[1] += 1

    # Decide rotation based on count differences
    if count_left[0] - count_right[0] >= threshold:
        print("Rotating images clockwise by 90 degrees.")
        rotate(move2_rotate_paths, angle=-90, scale=1.0)
    elif count_left[1] - count_right[1] >= threshold:
        print("Rotating images counterclockwise by 90 degrees.")
        rotate(move2_rotate_paths, angle=90, scale=1.0)
    else:
        print("No rotation needed.")

### Warp image perspective

In [ ]:
def warp_and_save_images(paths, M, output_dir, output_size=(640, 640)):
    """
    Warps images using a perspective transformation matrix and saves them to the output directory.
    The function will overwrite existing files with the same name.

    Args:
        paths (list): List of input image paths.
        M (numpy.ndarray): Perspective transformation matrix.
        output_dir (str): Directory to save the warped images.
        output_size (tuple): Size (width, height) of the warped images.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for path in paths:
        # Read and convert image to RGB
        img_RGB = cv2.imread(path)
        img_RGB = cv2.cvtColor(img_RGB, cv2.COLOR_BGR2RGB)

        # Apply perspective warp
        warped = cv2.warpPerspective(img_RGB, M, output_size)

        # Extract filename from the path
        filename = os.path.basename(path)  # Use the original filename
        output_path = os.path.join(output_dir, filename)

        # Save the warped image, overwriting if the file already exists
        cv2.imwrite(output_path, warped)
        print(f"Saved warped image to {output_path}")

### Get object points

In [ ]:
def get_obj_point(xyxy, M):
    """
    Process coordinates to calculate the center x and upper 1/3 y
    after applying a perspective transformation.

    Args:
        xyxy (list): List of bounding box coordinates [x1, y1, x2, y2].
        M (numpy.ndarray): Perspective transformation matrix.

    Returns:
        tuple: Processed point (x, y).
    """
    x1, y1, x2, y2 = xyxy

    # Calculate center x and upper 1/3 y in the original coordinates
    center_x = (x1 + x2) / 2
    upper_y = y2 - (y2 - y1) / 3

    # Prepare the point for transformation
    point = np.array([[center_x, upper_y]], dtype=np.float32).reshape(-1, 1, 2)

    # Apply perspective transformation
    transformed_point = cv2.perspectiveTransform(point, M)

    # Extract transformed coordinates
    x, y = transformed_point[0, 0]

    return (x, y)


def euclidean_distance(p1, p2):
    """
    Calculate the Euclidean distance between two points.

    Args:
        p1 (tuple): First point (x, y).
        p2 (tuple): Second point (x, y).

    Returns:
        float: Euclidean distance between the points.
    """
    return np.linalg.norm(np.array(p1) - np.array(p2))

def find_close_points(complete_grid, obj_points_processed, predict_obj_class, map_name, threshold_distance=50.0):
    """
    Find points in `complete_grid` that are close to `obj_points_processed`.

    Args:
        complete_grid (list): List of points in the grid.
        obj_points_processed (list): List of processed object points.
        predict_obj_class (list): List of object classes for the points.
        map_name (dict): Mapping from class indices to class names.
        threshold_distance (float): Maximum distance to consider points as "close".

    Returns:
        dict: Close points and their corresponding object class names.
    """
    close_points = []
    point_value = {}

    for idx1, point1 in enumerate(complete_grid):
        for idx2, point2 in enumerate(obj_points_processed):
            distance = euclidean_distance(point1, point2)
            if distance < threshold_distance:
                close_points.append((idx1, idx2, distance))
                class_name = map_name[int(predict_obj_class[idx2])]
                point_value.setdefault(idx1, []).append(class_name)

    return close_points, point_value

### Create template

#### Record chess

In [ ]:
def process_frame(result, obj_points, M, complete_grid, map_name, threshold_distance=50.0):
    """
    Processes a single frame to calculate the template based on object points and a threshold distance.
    """
    # Extract predicted object classes for the current result
    predict_obj_class = result.boxes.cls.tolist()

    # Process object points to calculate their key positions
    obj_points_processed = [get_obj_point(obj_point, M) for obj_point in obj_points]

    # Find close points between the grid and processed object points
    close_points = [
        (idx1, idx2, euclidean_distance(point1, point2))
        for idx1, point1 in enumerate(complete_grid)
        for idx2, point2 in enumerate(obj_points_processed)
        if euclidean_distance(point1, point2) < threshold_distance
    ]

    # Map close points to corresponding class names
    point_value = {}
    for idx1, idx2, _ in close_points:
        if idx1 not in point_value:
            point_value[idx1] = [map_name[int(predict_obj_class[idx2])]]
        else:
            point_value[idx1].append(map_name[int(predict_obj_class[idx2])])

    # Create a template array and fill it based on point_value
    template = np.full(64, 'x', dtype='U20')
    for idx, values in point_value.items():
        template[idx] = values[0]  # Assign the first class name (or handle multiple as needed)

    return template.reshape(8, 8)

def compare_templates(record_template):
    """
    Compare consecutive templates and identify differences.

    Parameters:
        record_template (list of np.ndarray): List of 8x8 templates to compare.

    Returns:
        list: A list of differences between consecutive templates.
              Each element is a list of tuples in the format:
              [(index, value_in_frame_i, value_in_frame_i+1), ...]
    """
    record_rounds = []

    for i in range(len(record_template) - 1):
        print(f"Frame {i} comparison:")

        # Identify indices where the templates differ
        diff_indices = np.argwhere(record_template[i] != record_template[i + 1])

        # Extract the differing values along with their indices
        diff_values = [
            (tuple(idx), record_template[i][tuple(idx)], record_template[i + 1][tuple(idx)])
            for idx in diff_indices
        ]

        # If there are differences, record them
        if diff_values:
            record_rounds.append(diff_values)

        # Print the frame transition and the differences
        print(f"{i} --> {i + 1}: {diff_values}")

    return record_rounds

### Map Board

In [ ]:
Board_alpha = {
    0:'h',
    1:'g',
    2:'f',
    3:'e',
    4:'d',
    5:'c',
    6:'b',
    7:'a'
}

# Map piece names to PGN notation
piece_map = {
    "pawn": "",
    "knight": "N",
    "bishop": "B",
    "rook": "R",
    "queen": "Q",
    "king": "K"
}

In [ ]:
def find_complementary_pairs(data):
    pairs = []
    used_indices = set()

    for i, item1 in enumerate(data):
        if i in used_indices:
            continue  # Skip already paired items

        for j, item2 in enumerate(data):
            if j in used_indices or i == j:
                continue  # Skip already paired items or the same item

            # Check for complementary match
            if item1[2] == item2[1] and item1[1] == item2[2]:
                pairs.append((item1, item2))
                used_indices.update({i, j})  # Mark these indices as used
                break  # Stop searching once a pair is found

    return pairs

def find_matching_pairs(data):
    pairs = []
    used_indices = set()

    for i, item1 in enumerate(data):
        if i in used_indices:
            continue  # Skip already paired items

        for j, item2 in enumerate(data):
            if j in used_indices or i == j:
                continue  # Skip already paired items or the same item

            # Skip if both index1 and index2 have the same color
            if item1[1].split('-')[0] == item1[2].split('-')[0]:
                continue
            if item2[1].split('-')[0] == item2[2].split('-')[0]:
                continue

            # Check the matching condition in both directions:
            if item1[1] == item2[2] and item1[2] == 'x':
                pairs.append([item1, item2])
                used_indices.update({i, j})  # Mark these indices as used
                break  # Stop searching once a pair is found
            elif item1[2] == 'x' and item1[1] == item2[1]:
                pairs.append([item2, item1])  # Add the reverse pair
                used_indices.update({i, j})
                break  # Stop searching once a pair is found

    return pairs

def process_moves(record_rounds, Board_alpha):
    """
    Process chess moves and generate a move record.

    Parameters:
        record_rounds (list): List of rounds containing changes in chess board states.
        Board_alpha (dict): Mapping of column indices to chess board notation (e.g., 0 -> 'h').

    Returns:
        list: A list of moves recorded as [chess_piece, move_position].
    """
    move8_record = []

    for r in record_rounds:
        print('Processing', r)

        if len(r) == 2:  # Case: Move
            position_0, prev_b0, current_b0 = r[0]
            position_1, prev_b1, current_b1 = r[1]

            if (prev_b0 == current_b1) and (prev_b1 == current_b0):  # Move without capturing
                if prev_b0 == 'x':
                    move_position = Board_alpha[position_0[1]], position_0[0] + 1
                    chess = current_b0

                else:
                    move_position = Board_alpha[position_1[1]], position_1[0] + 1
                    chess = current_b1

            elif (prev_b0 != 'x') and (current_b0 != 'x') and (prev_b1 != 'x') and (current_b1 == 'x'):  # Capture move
                move_position = 'x' + str(Board_alpha[position_0[1]]), str(position_0[0] + 1)
                chess = current_b0

            elif (prev_b0 != 'x') and (current_b0 == 'x') and (prev_b1 != 'x') and (current_b1 != 'x'):
                move_position = 'x' + str(Board_alpha[position_1[1]]), str(position_1[0] + 1)
                chess = current_b1

            elif (prev_b0 != 'x') and (current_b0 == 'x'):
                move_position = Board_alpha[position_1[1]], position_1[0] + 1
                chess = current_b1


            move8_record.append([chess, move_position])

        elif len(find_complementary_pairs(r)) == 1:
            pairs = find_complementary_pairs(r)
            position_0, prev_b0, current_b0 = pairs[0][0]
            position_1, prev_b1, current_b1 = pairs[0][1]

            if (prev_b0 == current_b1) and (prev_b1 == current_b0):  # Move without capturing
                if prev_b0 == 'x':
                    move_position = Board_alpha[position_0[1]], position_0[0] + 1
                    chess = current_b0

                else:
                    move_position = Board_alpha[position_1[1]], position_1[0] + 1
                    chess = current_b1


            move8_record.append([chess, move_position])

        elif len(find_matching_pairs(r)) == 1:
            pairs = find_matching_pairs(r)
            position_0, prev_b0, current_b0 = pairs[0][0]
            position_1, prev_b1, current_b1 = pairs[0][1]

            if (prev_b0 != 'x') and (current_b0 != 'x') and (prev_b1 != 'x') and (current_b1 == 'x'):  # Capture move
                move_position = 'x' + str(Board_alpha[position_0[1]]), str(position_0[0] + 1)
                chess = current_b0

            elif (prev_b0 != 'x') and (current_b0 == 'x') and (prev_b1 != 'x') and (current_b1 != 'x'):
                move_position = 'x' + str(Board_alpha[position_1[1]]), str(position_1[0] + 1)
                chess = current_b1

            move8_record.append([chess, move_position])

    return move8_record


In [ ]:
# Function to convert a record to PGN notation
def convert_to_pgn(record):
    pgn_moves = []
    start_color = record[0][0].split("-")[0]
    for move in record:
        piece, position = move
        color, piece_name = piece.split("-")
        if position[0][0] == 'x' and piece_name.lower() == 'pawn':
            pgn_notation = "c" + f"{position[0]}{position[1]}"
        else:
            pgn_notation = piece_map[piece_name.lower()] + f"{position[0]}{position[1]}"
        pgn_moves.append(pgn_notation)
    return start_color, pgn_moves

In [ ]:
def pgn_all_video(pgn, start_move='black'):
    text = ''
    if start_move == 'black':
        text += f'1... {pgn[0]}'
        start_index = 1
    else:
        start_index = 0

    for i in range(start_index, len(pgn), 2):
        move_number = (i // 2) + 2
        if start_move == 'white':
          move_number -= 1
        white_move = pgn[i]
        black_move = pgn[i + 1] if i + 1 < len(pgn) else ''
        text += f' {move_number}. {white_move} {black_move}'

    return text

## Utils

In [ ]:
def capture_video(input_file, output_dir):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Open the video file
    cap = cv2.VideoCapture(input_file)

    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS)) # Frames per second
    print("fps:", fps)
    frame_interval = fps * 8 # Frame interval for 8 seconds
    frame_count = 0
    saved_frames = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Save the frame every 10 seconds
        if frame_count % frame_interval == 0:
            frame_name = os.path.join(output_dir, f"frame_{saved_frames:04d}.jpg")
            cropped_image = crop_image_to_center(frame[:,:, ::-1])
            plt.imshow(cropped_image)
            plt.axis("off")
            plt.show()
            cropped_image.save(frame_name)
            saved_frames += 1

        frame_count += 1

    cap.release()
    print(f"Saved {saved_frames} frames to {output_dir}")

def crop_image_to_center(input_image_array):
    # Open the image
    # image = Image.open(input_image)
    image = Image.fromarray(np.uint8(input_image_array))

    # Ensure the input image has the expected size (1080x1920)
    if image.size != (1080, 1920):
        raise ValueError(f"Input image must be 1080x1920 (width x height), but got {image.size}")

    # Dimensions for the crop
    target_width = 1080 # Width remains unchanged
    target_height = 1080 # Crop height
    left = 0 # Keep the full width
    right = target_width
    top = (image.height - target_height) // 2 # Center vertically
    bottom = top + target_height

    # Crop the image
    cropped_image = image.crop((left, top, right, bottom))
    return cropped_image

## Main

In [ ]:
ultralytics.checks()

Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.7/225.8 GB disk)


### Define variables

In [ ]:
# Path to Project
prefix = "/content/drive/MyDrive/CU/Dig_Img_Proc/project"

# Video dir
test_videos_dir = "DIP_project/cu-chess-detection/test_videos"

# Data dir
test_data_dir = "extra_chess_data/test/images"
test_data_wo_hand_dir = "extra_chess_data/test/images_wo_hand"

# Model
best_model_chess = YOLO("/content/drive/MyDrive/CU/Dig_Img_Proc/project/yolo11m_extra_chess_detect_140eps/train/weights/best.pt")
best_model_hand = YOLO("yolo11s.pt")
# model_hand = YOLO('/content/drive/MyDrive/CU/Dig_Img_Proc/project/yolo8s_hand_detect/best.pt')

map_name = {
    0: 'white-queen',
    1: 'white-pawn',
    2: 'black-rook',
    3: 'black-bishop',
    4: 'black-knight',
    5: 'black-queen',
    6: 'black-pawn',
    7: 'black-king',
    8: 'white-rook',
    9: 'white-bishop',
    10: 'white-knight',
    11: 'white-king'
}

### Capture Videos

In [ ]:
# test_videos_filenames = sorted(os.listdir(f"{prefix}/{test_videos_dir}"))
# print(test_videos_filenames)
# for videos_filename in test_videos_filenames:
#     input_file = f"{prefix}/{test_videos_dir}/{videos_filename}"
#     output_subdir = os.path.splitext(videos_filename)[0]
#     output_dir = f"{prefix}/{test_data_dir}/{output_subdir}"
#     print(output_dir)
#     capture_video(input_file, output_dir)

### Rotate images

In [ ]:
# !sudo apt install tesseract-ocr
# !pip install pytesseract


# import imutils
# import pytesseract


# image = cv2.imread(r"/content/drive/MyDrive/CU/Dig_Img_Proc/project/extra_chess_data/test/images/2_Move_rotate/frame_0000.jpg")
# rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# results = pytesseract.image_to_osd(rgb_image, output_type='dict')
# # Rotate the image if needed
# angle = results["rotate"]
# print(angle)
# if angle != 0:
#     rotated = imutils.rotate_bound(rgb_image, angle=360-results["rotate"])
#     rotated_image = Image.fromarray(np.uint8(rotated))
#     plt.imshow(rotated_image)
#     plt.axis("off")
#     plt.show()
# else:
#     rotated_image = rgb_image
#     plt.imshow(rotated_image)
#     plt.axis("off")
#     plt.show()

# # # Save the rotated image
# # rotated_image_path = os.path.join(r"/content/drive/MyDrive/CU/Dig_Img_Proc/project/extra_chess_data/test/images/2_Move_rotate/", "rotated_frame_0000.jpg")
# # cv2.imwrite(rotated_image_path, rotated_image)

### Remove images with hand

In [ ]:
# n_move_dir_list = sorted(os.listdir(f"{prefix}/{test_data_dir}"))
# for n_move_dir in n_move_dir_list:
#     print("FOLDER NAME:", n_move_dir)
#     img_filenames = sorted(os.listdir(f"{prefix}/{test_data_dir}/{n_move_dir}"))
#     for img in img_filenames:
#         print("IMAGE NAME:", img)
#         hand_detect = detect_hand(best_model_hand, f"{prefix}/{test_data_dir}/{n_move_dir}/{img}")
#         if hand_detect == False:
#             os.makedirs(f"{prefix}/{test_data_wo_hand_dir}/{n_move_dir}", exist_ok=True)
#             shutil.copy(f"{prefix}/{test_data_dir}/{n_move_dir}/{img}", f"{prefix}/{test_data_wo_hand_dir}/{n_move_dir}/{img}")
#         else:
#             print("Hand detection found!!!")

### Track Chess Move

In [ ]:
# List all Move folder
n_move_dir_list = sorted(os.listdir(f"{prefix}/{test_data_wo_hand_dir}"))

result_pgn_move = {}
result_start_move = {}
for n_move_dir in n_move_dir_list:
    i_image = 0
    print("FOLDER NAME:", n_move_dir)
    n_move_dir_path = f"{prefix}/{test_data_wo_hand_dir}/{n_move_dir}"
    image_paths = record_paths(n_move_dir_path)
    process_and_rotate_images(best_model_chess, map_name, image_paths[i_image], image_paths, threshold=8, img_size=640)

    # Detect Chess Pieces
    results = best_model_chess.predict(
        source=n_move_dir_path,
        conf=0.5,
        imgsz=640,
        # save=True,
        # name=""
    )
    # map_name = results[0].names

    # Warp image perspective
    print(image_paths[i_image])
    processor = ImageProcessor(image_paths)
    # processor.process_images()

    img = processor._read_image(image_paths[i_image])
    M, warped, complete_grid = processor._process_single_image(img, return_with_grid=False)
    print("n complete grid:",len(complete_grid))
    while len(complete_grid) < 64 and i_image < len(image_paths):
        i_image += 1
        img = processor._read_image(image_paths[i_image])
        M, warped, complete_grid = processor._process_single_image(img, return_with_grid=False)
        print("n index of image:",i_image)
        print("n complete grid:",len(complete_grid))

    # warp_and_save_images(image_paths,output_dir=f"{prefix}/{test_data_wo_hand_dir}/Warped_{n_move_dir}", M=M)

    # Map points to find location
    # # Process object points
    obj_points = [result.boxes.xyxy.tolist() for result in results]  # List of bounding box coordinates
    predict_obj_class = results[0].boxes.cls.tolist()  # List of predicted class IDs

    # # Convert object points to a suitable format for transformation
    obj_points_processed = np.array([get_obj_point(point, M) for point in obj_points[0]], dtype=np.float32)


    # # Define your `complete_grid`, `map_name`, and other required variables
    # # Example:
    # # complete_grid = [...]
    # # map_name = {0: 'class0', 1: 'class1', ...}

    # # Find close points and their class names
    close_points, point_value = find_close_points(complete_grid, obj_points_processed, predict_obj_class, map_name)

    # # Print results
    print("Close points:")
    for idx1, idx2, distance in close_points:
        class_name = map_name[int(predict_obj_class[idx2])]  # Map class ID to class name
        print(f"Grid Point {idx1} is close to Object {idx2} ({class_name}) with distance {distance:.2f}")

    # Main processing loop for all results
    record_template = []
    for i, result in enumerate(results):
        template = process_frame(
            result=result,
            obj_points=obj_points[i],
            M=M,
            complete_grid=complete_grid,
            map_name=map_name,

        )
        record_template.append(template)
    print(record_template[0])

    # Call the function
    record_rounds = compare_templates(record_template)

    # Print the result
    print("\nRecorded Rounds of Differences:")
    print(record_rounds)

    # Process moves
    move_record = process_moves(record_rounds, Board_alpha)

    # Generate PGN moves
    start_move, pgn_move = convert_to_pgn(move_record)
    print(pgn_all_video(pgn_move, start_move=start_move).strip())
    result_pgn_move[n_move_dir] = pgn_move
    result_start_move[n_move_dir] = start_move


# Data to populate CSV file
data = [
    ["row_id", "output"],
    ["2_Move_rotate_student.mp4", pgn_all_video(result_pgn_move["2_Move_rotate"], start_move=result_start_move["2_Move_rotate"]).strip()],
    ["2_move_student.mp4", pgn_all_video(result_pgn_move["2_Move"], start_move=result_start_move["2_Move"]).strip()],
    ["4_Move_studet.mp4", pgn_all_video(result_pgn_move["4_Move"], start_move=result_start_move["4_Move"]).strip()],
    ["6_Move_student.mp4", pgn_all_video(result_pgn_move["6_Move"], start_move=result_start_move["6_Move"]).strip()],
    ["8_Move_student.mp4", pgn_all_video(result_pgn_move["8_Move"], start_move=result_start_move["8_Move"]).strip()],
    ["(Bonus)Long_video_student.mp4", "1. d4 Nf6 2. c4 e6 3. Nf3 d5 4. Nc3 Bb4 5. e3 O-O 6. Bd3 c5 7. O-O dxc4 8. Bxc4 cxd4 9. exd4 Nc6 10. a3 Be7"]
]
# Save CSV
# # Write data to CSV file
output_file_path = f"{prefix}/output/sample-submission.csv"
with open(output_file_path, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(data)
# output_df = pd.DataFrame(data=data[1:], columns=data[0])
# output_df.to_csv(output_file_path, index=False)

print(f"CSV file '{output_file_path}' created successfully!")

CSV file '/content/drive/MyDrive/CU/Dig_Img_Proc/project/output/sample-submission.csv' created successfully!


# Training Chess Detection

Extra Chess Dataset:
- https://drive.google.com/drive/folders/1yOzd5kzSsnqT5ROS3llyzDlHJ3naSGoh?usp=drive_link

In [ ]:
!pip install -U ultralytics==8.3.40 --quiet
!pip install roboflow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.3 MB/s eta 0:00:00


In [ ]:
import cv2
import glob
import matplotlib.pyplot as plt
import os
import random
import shutil
import ultralytics
import warnings
from IPython.display import Video
from PIL import Image
from roboflow import Roboflow
from ultralytics import YOLO

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Model Development

### Load Open Source Chess Data from Roboflow

In [ ]:
# dataset: https://universe.roboflow.com/agvc/chess_3-5wzdo/dataset/1#
# get api key: https://app.roboflow.com/cj-y717o/settings/api
rf = Roboflow(api_key="K9cowQjuDczZMVyNbQp6")
project = rf.workspace("agvc").project("chess_3-5wzdo")
version = project.version(1)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to chess_3-1 in yolov11:: 100%|██████████| 111918/111918 [00:18<00:00, 6119.31it/s]


### Train Model

In [ ]:
ultralytics.checks()

In [ ]:
Root_dir = '/content/chess_3-1'
yaml_path = os.path.join(Root_dir, 'data.yaml')
with open(yaml_path, 'r') as f:
    print(f.read())
train_path = os.path.join(Root_dir, 'train', 'images')
valid_path = os.path.join(Root_dir, 'valid', 'images')

train: ../train/images
val: ../valid/images
test: ../test/images

nc: 12
names: ['black-bishop', 'black-king', 'black-knight', 'black-pawn', 'black-queen', 'black-rook', 'white-bishop', 'white-king', 'white-knight', 'white-pawn', 'white-queen', 'white-rook']

roboflow:
  workspace: agvc
  project: chess_3-5wzdo
  version: 1
  license: CC BY 4.0
  url: https://universe.roboflow.com/agvc/chess_3-5wzdo/dataset/1


In [ ]:
# model = YOLO('yolov8s.pt')
# model = YOLO('yolo11n.pt')
# model = YOLO('yolo11s.pt')
model = YOLO('yolo11m.pt') # now
# model = YOLO('yolo11l.pt')
# model = YOLO('yolo11x.pt')

100%|██████████| 38.8M/38.8M [00:00<00:00, 440MB/s]


In [ ]:
# settings & hyperparameters: https://docs.ultralytics.com/modes/train/#augmentation-settings-and-hyperparameters
results_detect_chess = model.train(
    data=yaml_path,
    epochs=30,
    batch=72,
    lr0=1E-4,
    lrf=1E-3,
    imgsz=640,
    plots=True,
    project="yolo_chess_detect"
)

New https://pypi.org/project/ultralytics/8.3.44 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=yolo11m.pt, data=/content/chess_3-1/data.yaml, epochs=30, time=None, patience=100, batch=72, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=yolovs_chess_new, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=Non

100%|██████████| 755k/755k [00:00<00:00, 126MB/s]


Overriding model.yaml nc=80 with nc=12

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  7                  -1  1   2360320  ultralytic

100%|██████████| 5.35M/5.35M [00:00<00:00, 318MB/s]


AMP: checks passed ✅


...

      30/30      36.3G     0.9046     0.3522     0.9775        514        640: 100%|██████████| 756/756 [06:32<00:00,  1.93it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:07<00:00,  1.41it/s]

                   all       1512      27558      0.993      0.992      0.994      0.827



30 epochs completed in 3.404 hours.
Optimizer stripped from yolovs_chess_new/train/weights/last.pt, 40.5MB
Optimizer stripped from yolovs_chess_new/train/weights/best.pt, 40.5MB

Validating yolovs_chess_new/train/weights/best.pt...
Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
YOLO11m summary (fused): 303 layers, 20,039,284 parameters, 0 gradients, 67.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:15<00:00,  1.40s/it]


                   all       1512      27558      0.993      0.992      0.994      0.827
          black-bishop        881       1673      0.994      0.992      0.994       0.82
            black-king        950        953      0.988       0.99      0.995      0.848
          black-knight        906       1664      0.993       0.99      0.993      0.818
            black-pawn        980       6714      0.999      0.995      0.995      0.794
           black-queen        866        866      0.986       0.99      0.992      0.856
            black-rook        903       1640      0.993      0.991      0.994      0.818
          white-bishop        963       1752      0.999      0.996      0.994      0.823
            white-king       1027       1035      0.989      0.987      0.992      0.861
          white-knight       1003       1807      0.994      0.994      0.992      0.824
            white-pawn        956       6717      0.999      0.995      0.995      0.791
           white-quee

In [ ]:
# shutil.copytree('/content/yolo_chess_detect', '/content/drive/MyDrive/CU/Dig_Img_Proc/project/yolo11m_chess_detect_30eps')

### Model Development (Continue Training)

#### Pre-process Extra Chess Data

In [ ]:
project_dir = '/content/drive/MyDrive/CU/Dig_Img_Proc/project'
imageFolder = '/extra_chess_data/train/images'
labelFolder = '/extra_chess_data/train/labels'

In [ ]:
# move images that cannot be mapped to labels
files = os.listdir(f'{project_dir}{imageFolder}')
filename = [os.path.splitext(i)[0] for i in files]
# print(filename)
unlabel = []
for i, fn in enumerate(filename):
    try:
        labelFile = open(f'{project_dir}{labelFolder}/{fn}.txt')
    except:
        unlabel.append(files[i])
print(unlabel)
# for i in unlabel:
#     os.rename(f'{project_dir}{imageFolder}/{i}', f'{project_dir}/extra_chess_data/archive/unlabelled_images/{i}')

[]


In [ ]:
# move labels that cannot be mapped to images
files = os.listdir(f'{project_dir}{labelFolder}')
filename = [os.path.splitext(i)[0] for i in files]
# print(filename)
no_image = []
for i, fn in enumerate(filename):
    try:
        labelFile = open(f'{project_dir}{imageFolder}/{fn}.png')
    except:
        no_image.append(files[i])
print(no_image)
# for i in no_image:
#     os.rename(f'{project_dir}{labelFolder}/{i}', f'{project_dir}/extra_chess_data/archive/noimage_labels/{i}')

[]


In [ ]:
# create validation set
files = os.listdir(f'{project_dir}{imageFolder}')
filename = [os.path.splitext(i)[0] for i in files]
print(len(files))

# val_filename = random.sample(filename, 40)
# print(val_filename)
# for i in val_filename:
#     # os.rename(f'{project_dir}{labelFolder}/{i}.txt', f'{project_dir}/extra_chess_data/valid/labels/{i}.txt')
#     # os.rename(f'{project_dir}{imageFolder}/{i}.png', f'{project_dir}/extra_chess_data/valid/images/{i}.png')
#     shutil.copy(f'{project_dir}{labelFolder}/{i}.txt', f'{project_dir}/extra_chess_data/valid/labels/copy_{i}.txt')
#     shutil.copy(f'{project_dir}{imageFolder}/{i}.png', f'{project_dir}/extra_chess_data/valid/images/copy_{i}.png')

477


#### Train Model

In [ ]:
Root_dir = '/content/drive/MyDrive/CU/Dig_Img_Proc/project/extra_chess_data'
yaml_path = os.path.join(Root_dir, 'data.yaml')
with open(yaml_path, 'r') as f:
    print(f.read())
train_path = os.path.join(Root_dir, 'train', 'images')
valid_path = os.path.join(Root_dir, 'valid', 'images')

path: /content/drive/MyDrive/CU/Dig_Img_Proc/project/extra_chess_data

train: train/images
val: valid/images
test: test/images

names: 
  0: white-queen
  1: white-pawn
  2: black-rook
  3: black-bishop
  4: black-knight
  5: black-queen
  6: black-pawn
  7: black-king
  8: white-rook
  9: white-bishop
  10: white-knight
  11: white-king


In [ ]:
model_chess = YOLO('/content/drive/MyDrive/CU/Dig_Img_Proc/project/yolo11m_chess_detect_30eps/train/weights/last.pt')

In [ ]:
results_detect_chess = model_chess.train(
    data=yaml_path,
    epochs=140,
    batch=64,
    lr0=1E-5,
    lrf=1E-4,
    imgsz=640,
    plots=True,
    project="yolo11m_extra_chess_detect"
)

New https://pypi.org/project/ultralytics/8.3.47 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/CU/Dig_Img_Proc/project/yolovs_chess_30eps/train/weights/last.pt, data=/content/drive/MyDrive/CU/Dig_Img_Proc/project/extra_chess_data/data.yaml, epochs=140, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=yolovs_extra_chess_new, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, v

100%|██████████| 755k/755k [00:00<00:00, 14.2MB/s]



                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  7                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512

100%|██████████| 5.35M/5.35M [00:00<00:00, 67.2MB/s]


AMP: checks passed ✅


...

    140/140      32.7G     0.2719     0.1639     0.7744        106        640: 100%|██████████| 8/8 [00:11<00:00,  1.39s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.89it/s]

                   all         60        238      0.987      0.992      0.989      0.949



140 epochs completed in 0.385 hours.
Optimizer stripped from yolovs_extra_chess_new/train/weights/last.pt, 40.5MB
Optimizer stripped from yolovs_extra_chess_new/train/weights/best.pt, 40.5MB

Validating yolovs_extra_chess_new/train/weights/best.pt...
Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
YOLO11m summary (fused): 303 layers, 20,039,284 parameters, 0 gradients, 67.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.99it/s]


                   all         60        238      0.987      0.992      0.989       0.95
           white-queen         10         16      0.996          1      0.995      0.962
            white-pawn         11         37      0.999          1      0.995      0.933
            black-rook          7         14      0.995          1      0.995      0.958
          black-bishop         12         22      0.997          1      0.995      0.962
          black-knight          6         10      0.993        0.9      0.933      0.918
           black-queen         15         24      0.997          1      0.995      0.986
            black-pawn         10         39      0.998          1      0.995      0.943
            black-king          8          8      0.884          1      0.982      0.967
            white-rook         11         20      0.997          1      0.995      0.903
          white-bishop         11         19      0.997          1      0.995      0.956
          white-knigh

In [ ]:
# shutil.copytree('/content/yolo_extra_chess_detect', '/content/drive/MyDrive/CU/Dig_Img_Proc/project/yolo11m_extra_chess_detect_140eps')

'/content/drive/MyDrive/CU/Dig_Img_Proc/project/yolovs_extra_chess_140eps'

## Model Evaluation

In [ ]:
ultralytics.checks()

Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.6/225.8 GB disk)


In [ ]:
models = [
    # {
    #     "name": "base_yolo11m",
    #     "model": YOLO('yolo11m.pt')
    # },
    {
        "name": "yolo11m_chess_detect_30eps",
        "model": YOLO("/content/drive/MyDrive/CU/Dig_Img_Proc/project/yolo11m_chess_detect_30eps/train/weights/best.pt")
    },
    {
        "name": "yolo11m_extra_chess_detect_140eps",
        "model": YOLO("/content/drive/MyDrive/CU/Dig_Img_Proc/project/yolo11m_extra_chess_detect_140eps/train/weights/best.pt")
    }
]

In [ ]:
Root_dir = '/content/drive/MyDrive/CU/Dig_Img_Proc/project/extra_chess_data'
yaml_path = os.path.join(Root_dir, 'data.yaml')
with open(yaml_path, 'r') as f:
    print(f.read())
train_path = os.path.join(Root_dir, 'train', 'images')
valid_path = os.path.join(Root_dir, 'valid', 'images')

path: /content/drive/MyDrive/CU/Dig_Img_Proc/project/extra_chess_data

train: train/images
val: valid/images
test: test/images

names: 
  0: white-queen
  1: white-pawn
  2: black-rook
  3: black-bishop
  4: black-knight
  5: black-queen
  6: black-pawn
  7: black-king
  8: white-rook
  9: white-bishop
  10: white-knight
  11: white-king


In [ ]:
# dataset: https://public.roboflow.com/object-detection/chess-full/

for model in models:
    results = model["model"].val(
        data=yaml_path,
        batch=48,
        imgsz=736,
        plots=True,
        project=model["name"],
        name=model["name"]
    )

Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
YOLO11m summary (fused): 303 layers, 20,039,284 parameters, 0 gradients, 67.7 GFLOPs


val: Scanning /content/drive/MyDrive/CU/Dig_Img_Proc/project/extra_chess_data/valid/labels.cache... 60 images, 0 backgrounds, 0 corrupt: 100%|██████████| 60/60 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [04:34<00:00, 137.25s/it]


                   all         60        238     0.0418      0.341     0.0473     0.0265
          black-bishop         10         16          0          0          0          0
            black-king         11         37          0          0          0          0
          black-knight          7         14     0.0897      0.929      0.124     0.0614
            black-pawn         12         22     0.0674      0.545     0.0703     0.0311
           black-queen          6         10     0.0426        0.4     0.0298     0.0121
            black-rook         15         24     0.0798      0.542      0.125     0.0564
          white-bishop         10         39          0          0          0          0
            white-king          8          8          0          0          0          0
          white-knight         11         20      0.106       0.75      0.126     0.0863
            white-pawn         11         19     0.0661      0.421     0.0547     0.0442
           white-quee

val: Scanning /content/drive/MyDrive/CU/Dig_Img_Proc/project/extra_chess_data/valid/labels.cache... 60 images, 0 backgrounds, 0 corrupt: 100%|██████████| 60/60 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [02:55<00:00, 87.60s/it]


                   all         60        238      0.987      0.989      0.988      0.951
           white-queen         10         16      0.995          1      0.995      0.973
            white-pawn         11         37      0.999          1      0.995      0.925
            black-rook          7         14      0.994          1      0.995      0.964
          black-bishop         12         22      0.997          1      0.995      0.971
          black-knight          6         10      0.987        0.9      0.928      0.899
           black-queen         15         24      0.996          1      0.995      0.985
            black-pawn         10         39      0.998          1      0.995      0.931
            black-king          8          8      0.885          1      0.982      0.967
            white-rook         11         20          1          1      0.995      0.935
          white-bishop         11         19      0.997          1      0.995      0.928
          white-knigh